<a href="https://colab.research.google.com/github/DhanashreerNerkar/AI-ML-NLP/blob/main/Finetuning_Personal_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install peft
!pip install accelerate
!pip install bitsandbytes
!pip install transformers
! install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 48.2 MB/s eta 0:00:00
install: missing destination file operand after 'datasets'
Try 'install --help' for more information.


In [4]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=a1e34442db5847dc64f7440e19d1fc4a091a20d8b7b03103bc2e3a983ccde5d1
  Stored in directory: /root/.cache/pip/wheels/92/a8/b7/d8a067c31a74de9ca252bbe53dea5f896faabd25d55f541037
Successfully built GPUtil


In [5]:
import  torch
import GPUtil
import os

GPUtil.showUtilization()
if torch.cuda.is_available():
  print("GPU avaiable")
else:
  print("GPU not available")

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU avaiable


In [6]:
import torch
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training,LoraConfig, get_peft_model

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

In [7]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `Colab Notebook` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-cred

In [18]:
base_model_id="meta-llama/Llama-2-7b-chat-hf"
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
  	bnb_4bit_use_double_quant=True,
  	bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model=AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    token=True # Explicitly pass the Hugging Face token
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [19]:
!git clone https://github.com/poloclub/Fine-tuning-LLMs.git

Cloning into 'Fine-tuning-LLMs'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 14), reused 29 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 9.34 MiB | 19.48 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [23]:
train_dataset=load_dataset(
    "text",
    data_files={
        "train":["/content/Fine-tuning-LLMs/data/hawaii_wf_1.txt",
                 "/content/Fine-tuning-LLMs/data/hawaii_wf_2.txt"]},
    split="train"
)

Generating train split: 0 examples [00:00, ? examples/s]

In [35]:
train_dataset["text"][1]

'COUNTY OF MAUI'

In [31]:
tokenizer=LlamaTokenizer.from_pretrained(base_model_id,use_fast=False, trust_remote_code=True, add_eos_token=True)
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token':tokenizer.eos_token})

In [46]:
tokenized_train_dataset=[]
for phrase in train_dataset:
  tokenized_train_dataset.append(tokenizer(phrase["text"]))

In [47]:
tokenized_train_dataset[1]

{'input_ids': [1, 21122, 29979, 8079, 14861, 3120, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [48]:
tokenized_train_dataset[2]

{'input_ids': [1, 29871, 29945, 29945, 14861, 29950, 1964, 2190, 29902, 6850, 1525, 2544, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [49]:
tokenizer.eos_token

'</s>'

In [50]:
model.config.use_cache = False
model.gradient_checkpointing_enable()
model=prepare_model_for_kbit_training(model)
config=LoraConfig(
    r=8,
    lora_alpha=6,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)
model=get_peft_model(model,config)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [53]:
trainer=transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=transformers.TrainingArguments(
        output_dir="./FinetunedModel_Results",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=3,
        learning_rate=1e-4,
        max_steps=1000,
        bf16=False, # Corrected from bp16
        optim="paged_adamw_8bit", # Corrected from optium
        logging_dir="./log", # Corrected from loggin_dir
        save_strategy="epoch",
        save_steps=50,
        logging_steps=10),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer,mlm=False)
)
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,0.329900
20,0.331300
30,0.300500
40,0.312800
50,0.254600
60,0.263300
70,0.357100
80,0.308700
90,0.284100
100,0.292800


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/pyt

TrainOutput(global_step=1000, training_loss=0.2653747777938843, metrics={'train_runtime': 4445.5418, 'train_samples_per_second': 0.9, 'train_steps_per_second': 0.225, 'total_flos': 4493490217058304.0, 'train_loss': 0.2653747777938843, 'epoch': 26.31578947368421})

In [55]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, LlamaTokenizer

base_model_id="meta-llama/Llama-2-7b-chat-hf"
nf4Config=BitsAndBytesConfig(
    load_in_4bit=True,
  	bnb_4bit_use_double_quant=True,
  	bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer=LlamaTokenizer.from_pretrained(base_model_id,use_fast=False,trust_remote_code=True, add_eos_token=True)
base_model=AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=nf4Config,
    device_map="auto",
    trust_remote_code=True,
    token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [62]:
from peft import PeftModel

tokenizer=LlamaTokenizer.from_pretrained(base_model_id,use_fast=False,trust_remote_code=True)
modelFinetuned=PeftModel.from_pretrained(
    base_model,"./FinetunedModel_Results/checkpoint-38")

/usr/local/lib/python3.12/dist-packages/peft/peft_model.py:598: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.we

In [64]:
user_question="when did Hawaii wildfires start?"
eval_prompt= f"Question: {user_question} Just aswer this question accurately and concisely."
promptTokenizer=tokenizer(eval_prompt, return_tensors="pt").to("cuda")
modelFinetuned.eval()
with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenizer, max_new_tokens=1024)[0],skip_special_tokens=True))
  torch.cuda.empty_cache()

Question: when did Hawaii wildfires start? Just aswer this question accurately and concisely. nobody knows the answer?
Answer: Hawaii wildfires have been occurring for centuries, with the earliest recorded fire in 1775. However, the frequency and severity of wildfires in Hawaii have increased in recent years due to various factors such as climate change, drought, and human activity.

Here are some key points about the history of wildfires in Hawaii:

* 1775: The earliest recorded wildfire in Hawaii occurred on the Big Island, where a lightning strike ignited a fire that burned for several days.
* 1840s: Wildfires became more frequent and severe in Hawaii, with many fires caused by human activity such as farming and logging.
* 1930s: The construction of roads and other infrastructure in Hawaii's tropical forests led to an increase in wildfires, as well as the spread of invasive plant species that contributed to the fires.
* 1970s: The Hawaii Forest Reserve Act was established, which aim